In [20]:
import csv

torah = 'torah.csv'
megillot = 'chamesh megillot.csv'
navi1 = 'neviim rishonim.csv'
navi2 = 'neviim achronim.csv'
emet = 'sifrei emet.csv'
ketuvim = 'ketuvim.csv'


cant = ['֑','֒','֓','֔','֕','֖','֗','֘','֙','֚','֛','֜',
             '֝','֞','֠','֡','֢','֣','֤','֥','֦','֧','֨','֩','֪','֫','֬','֭','֯','׃']

aleph_bet = ['א','ב','ג','ד','ה','ו','ז','ח','ט','י','כ','ך','ל','מ','ם',
             'נ','ן','ס','ע','פ','ף','צ','ק','ר','ש','ת','װ','ױ','ײ','יִ',
             'ﬡ','ﬢ','ﬣ','ﬤ','ﬥ','ﬦ','ﬧ','ﬨ','שׁ','שׂ','שּׁ','שּׂ','אַ','אָ',
             'גּ','דּ','הּ','וּ','זּ','טּ','יּ','ךּ','כּ','לּ','מּ','נּ','סּ','ףּ',
             'פּ','צּ','שּ','תּ','וֹ','בֿ','כֿ','פֿ','ﭏ','בּ', 'קּ']

kamatz_katan = "ׇ"

meteg = "ֽ"


vowel = ['ְ','ֱ','ֲ','ֳ','ִ','ֵ','ֶ','ַ','ָ','ֹ','ֺ','ֻ','ּ','ֽ','־','ֿ','ׁ','ׂ','ׄ','ׅ','ׇ']

gutterals = ['א','ה','ח','ע','ר' ,'ﬡ','ﬣ','ﬧ']    

In [7]:
#removes trop but not vowels
def trop_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter not in cant and letter != meteg:
            no_cant_word = no_cant_word + letter
    return no_cant_word

In [57]:
def kamatz_katan_finder(bookset):
    #words_with_katan = {"כׇּל":[],"כׇל":[]}
    words_with_katan = []
    questionable_words = []
    
    with open(bookset, 'r', encoding='utf-8') as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        in_range = False
        for row in csvReader:
            row[4] = row[4].replace("{{מ:לגרמיה}}"," {{מ:לגרמיה}} ")
            row[4] = row[4].replace("{{מ:פסק}}"," {{מ:פסק}} ")
            words = row[4].split(" ")
            
            for word in words:
                word_snipped = word.replace("כׇל","")
                word_snipped = word_snipped.replace("כׇּל","")
                if kamatz_katan not in word_snipped:
                    continue
                word = word.replace("׃","")
                word_info = {}
                if "מ:קמץ" in word:
                    if "|ד=" in word:
                        start = word.index("|ד=")+3
                    else:
                        start = word.index("|")+1
                    if "|ס=" in word:
                        end = word.index("|",start)
                    else:
                        end = len(word)-2
                    word = word[start:end]
                    
                    word_info["question"] = True
                elif "ֳ" in word:
                    katan_loc = word.index("ׇ")
                    chataf_loc = word.index("ֳ")
                    if chataf_loc > katan_loc:
                        word_mini = word[katan_loc:chataf_loc]
                        counter = 0
                        letter = ""
                        for char in word_mini:
                            if char in aleph_bet:
                                counter += 1
                                letter = char
                        if counter == 1 and char in gutterals:
                            word_info["question"] = True
                        else: 
                            word_info["question"] = False
                else:
                    word_info["question"] = False
                word_info["word"] = word
                word_info["ref_he"] = row[3]
                word_info["ref_en"] = reference_parser(row[3])
                words_with_katan.append(word_info)
    return words_with_katan

In [65]:
sefer_names = {
    "בראשית":"Gen",
    "שמות":"Ex",
    "ויקרא":"Lev",
    "במדבר":"Num",
    "דברים":"Deut",
    "יהושע":"Josh",
    "שופטים":"Judges",
    "שמואל א":"1Sam",
    "שמואל ב":"2Sam",
    "מלכים א":"1Kings",
    "מלכים ב":"2Kings",
    "ישעיהו":"Is",
    "ירמיהו":"Jer",
    "יחזקאל":"Ezek",
    "הושע":"Hosea",
    "יואל":"Joel",
    "עמוס":"Amos",
    "עובדיה":"Obd",
    "יונה":"Jonah",
    "מיכה":"Micah",
    "נחום":"Nahum",
    "חבקוק":"Hab",
    "צפניה":"Zeph",
    "חגי":"Haggai",
    "זכריה":"Zek",
    "מלאכי":"Malachi",
    "שיר השירים":"Song of Songs",
    "רות":"Ruth",
    "אסתר":"Esther",
    "קהלת":"Eccl",
    "איכה":"Lam",
    "תהלים":"Psalms",
    "איוב":"Job",
    "משלי":"Prov"
}

letters_to_numbers = {
    "א":1,
    "ב":2,
    "ג":3,
    "ד":4,
    "ה":5,
    "ו":6,
    "ז":7,
    "ח":8,
    "ט":9,
    "י":10,
    "כ":20,
    "ל":30,
    "מ":40,
    "נ":50,
    "ס":60,
    "ע":70,
    "פ":80,
    "צ":90,
    "ק":100,
    "ר":200,
    "ש":300,
    "ת":400
}

In [48]:
def gematria_parser(string):
    result = 0
    for char in string:
        if char in letters_to_numbers.keys():
            result += letters_to_numbers[char]
    return result

In [44]:
def reference_parser(refstring):
    #{{מ:פסוק|בראשית|כ|יב}}
    book_start = refstring.index("פסוק|")+5
    book_end = refstring.index("|",book_start+1)-1
    book = refstring[book_start:book_end+1]
    chap_start = book_end+2
    chap_end = refstring.index("|",chap_start)
    chapter = refstring[chap_start:chap_end]
    verse_start = chap_end+1
    if "|" in refstring[verse_start:-1]:
        verse_end = refstring.index("|",verse_start+1)
    else:
        verse_end = refstring.index("}")
    verse = refstring[verse_start:verse_end]
    
    book_eng = sefer_names[book]
    chap_eng = gematria_parser(chapter)
    verse_eng = gematria_parser(verse)
    ref_eng = book_eng+" "+str(chap_eng)+":"+str(verse_eng)
    return ref_eng

In [45]:
 def exporter(katanlist):
    with open("output2.tex",'w',encoding='utf-8') as outfile:
        outfile.write("\documentclass[14pt]{article}\n")
        outfile.write("\\usepackage{kamatz_list_style}\n\\begin{document}\n")
        outfile.write("\\begin{itemize} ")
        for item in katanlist:
            if "{" in item:
                continue
            outfile.write("\item ")
            outfile.write(item["ref_en"])
            outfile.write("\hebword{")
            outfile.write(item["word"])
            outfile.write("}")
            if item["question"] == True:
                outfile.write(" This kamatz katan is subject to debate between experts and reading traditions.")
            outfile.write("\n\n")
        outfile.write("\end{itemize}\n\end{document} ")

In [66]:
kamatz_data_chumash = kamatz_katan_finder(emet)

In [67]:
exporter(kamatz_data_chumash)